# Rayleigh Waves Synthetic Auto-correlation Functions

In [ ]:
## Distributed python packages
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr

import pandas as pd
import time as ttime

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from math import radians, log
from datetime import datetime
from wmsan.synthetics import *

%matplotlib inline
__author__ = "Lisa Tomasetto"
__copyright__ = "Copyright 2024, UGA"
__credits__ = ["Lisa Tomasetto"]
__version__ = "2025.0.0"
__maintainer__ = "Lisa Tomasetto"
__email__ = "lisa.tomasetto@gmail.com"
__status__ = "Developpement"

## Make Nice Plots

In [ ]:
plt.style.use("ggplot")
SMALL_SIZE = 18
MEDIUM_SIZE = 22
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
matplotlib.rcParams['pdf.fonttype'] = 42

## Theory 
We will compute synthetic Cross-correlations between vertical components, using the formulation given in Tomasetto et al.(2024). Using the representation theorem and assuming spatially uncorrelated sources one can write the cross-correlation function between sensors A and B as:

$$ C(r_A, r_B, t) = \mathcal{FT}^{-1}\left[ \int_{\partial D} G(r_A, r, f) G^*(r_B, r, f) S(r,f) dr \right] $$

with:
- $G(r_A, r, f)$ the Green's function between a source in $ r $ and station A in $ r_A $. Here computed in AxiSEM.
- $S(r,f)$ the source PSD at position $ r $. Here given by the square of proxy for the source force amplitude computed in microseismic_sources.ipynb.
- $^*$ denotes the complex conjugate.
- $\partial D$ the domain of potential sources, here, the ocean's surface.
- $\mathcal{FT}^{-1}$ the Fourier transform.

#### References 

- [Nissen-Meyer, T., van Driel, M., Stähler, S. C., Hosseini, K., Hempel, S., Auer, L., ... & Fournier, A. (2014). AxiSEM: broadband 3-D seismic wavefields in axisymmetric media. Solid Earth, 5(1), 425-445.](https://se.copernicus.org/articles/5/425/2014/)

- [Tomasetto, L., Boué, P., Stehly, L., Ardhuin, F., & Nataf, H. C. (2024). On the stability of mantle‐sensitive P‐wave interference during a secondary microseismic event. Geophysical Research Letters, 51(8), e2023GL108018.](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2023GL108018)

## Create Synthetic Waveforms Database

In the following, we will use an AxiSEM archive file of seismic waveforms computed in ak135f model and sampled at 1 Hz.

First we will read and plot the archive.

[WARNING]
Please download the archive before running the following cells! 

Avalaible from: https://zenodo.org/records/11126562

Save in /ww3-source-maps/data/

In [ ]:
## Open Archive 
path_file_axisem = '../../data/NOISE_vertforce_dirac_0-ak135f_1.s_3600s.h5'
fe, time, N = get_synthetic_info(path_file_axisem=path_file_axisem, comp='Z')

distance = np.arange(0, 180.1, 0.1)
M = len(distance)
archive = np.zeros((M, N))

for i in range(M):
    archive[i,:] = open_axisem(distance[i], path_file_axisem=path_file_axisem, comp='Z')


In [ ]:
## Plot archive
plt.figure()
plt.pcolormesh(time, distance, archive,cmap='seismic', vmin=-10, vmax = 10)
plt.colorbar()
plt.xlabel('time')
plt.ylabel('distance')
plt.xlim(0, 3600)
plt.show()

## Tapering

Then we taper the synthetics waveforms between two given surface waves velocities.

### Radial Component

In [ ]:
## Taper
Umin = 2.5
Umax = 4.2

file_name_taper = '../../data/NOISE_vertforce_dirac_0-ak135f_1.s_3600_tapered_R.h5'

if os.path.isfile(file_name_taper):
    print('Tapered archive already exists')
    ## Open tapered archive
    h5_file = h5py.File(file_name_taper, 'r')
    tapered_archive = h5_file['WAVEFORMS'][:]
    h5_file.close()
    
else:
    tapered_archive = taper_axisem_archive(time, distance, archive_name=path_file_axisem, umin = Umin, umax=Umax, comp='R')

In [ ]:
## Plot Tapered archive 

plt.figure(figsize=(10, 5))
plt.pcolormesh(time, distance, tapered_archive, cmap='seismic', vmin=-10, vmax = 10)
plt.plot(distance*(np.pi*6371)/180/Umin, distance, 'r')
plt.plot(distance*(np.pi*6371)/180/Umax, distance, 'g')
plt.colorbar()
plt.xlabel('time')
plt.ylabel('distance')
plt.xlim(0, 3600)
plt.savefig('GF_archive.png', dpi=300, bbox_inches='tight')


In [ ]:
file_name_spectrum = '../../data/spectrum_archive_tapered_R.h5'

if os.path.isfile(file_name_spectrum):
    print('Spectrum archive already exists')
    ## Open Spectrum archive
    h5_file = h5py.File(file_name_spectrum, 'r')
    spectrum_archive_R = h5_file['SPECTRUM'][:]
    frequency = h5_file['frequency'][:]
    distance = h5_file['distance'][:]
    print(spectrum_archive_R.shape)
else:
    ## Create Spectrum archive from the tapered one
    create_spectrum_archive(time, distance, tapered_archive, file_name_spectrum)

### Vertical Component

In [ ]:
## Taper
Umin = 2500 # minimum Rayleigh wave velocity in m/s 
Umax = 4200 # minimum Rayleigh wave velocity in m/s 

file_name_taper = '../../data/NOISE_vertforce_dirac_0-ak135f_1.s_3600_tapered_Z.h5'

if os.path.isfile(file_name_taper):
    print('Tapered archive already exists')
    ## Open tapered archive
    h5_file = h5py.File(file_name_taper, 'r')
    tapered_archive = h5_file['WAVEFORMS'][:]
    h5_file.close()
    
else:
    tapered_archive = taper_axisem_archive(time, distance, archive_name=path_file_axisem, umin = Umin, umax=Umax, comp='Z')

In [ ]:
## Plot Tapered archive 

plt.figure(figsize=(10, 5))
plt.pcolormesh(time, distance, tapered_archive, cmap='seismic', vmin=-10, vmax = 10)
plt.plot(distance*(np.pi*6371)/180/Umin, distance, 'r')
plt.plot(distance*(np.pi*6371)/180/Umax, distance, 'g')
plt.colorbar()
plt.xlabel('time')
plt.ylabel('distance')
plt.xlim(0, 3600)
plt.savefig('GF_archive.png', dpi=300, bbox_inches='tight')

In [ ]:
file_name_spectrum = '../../data/spectrum_archive_tapered_Z.h5'

if os.path.isfile(file_name_spectrum):
    print('Spectrum archive already exists')
    ## Open Spectrum archive
    h5_file = h5py.File(file_name_spectrum, 'r')
    spectrum_archive_Z = h5_file['SPECTRUM'][:]
    frequency = h5_file['frequency'][:]
    distance = h5_file['distance'][:]
    print(spectrum_archive_Z.shape)
else:
    ## Create Spectrum archive from the tapered one
    create_spectrum_archive(time, distance, tapered_archive, file_name_spectrum)

## Synthetics Computation

## Spatial Extent

In [ ]:
## Spatial Exent
lon_min = 170
lon_max = -120
lat_min = 30
lat_max = 70

extent = [lon_min, lon_max, lat_min, lat_max]

## Station Pairs and Metadata

In [ ]:
## Station Pairs Metadata
id_sta = "OO.HYSB1.00"
lat_sta = 44.509791
lon_sta = -125.405258

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection=ccrs.Mollweide())
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='-', alpha=1)
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.LAND, color='linen', zorder=0)
ax.add_feature(cfeature.OCEAN, color='lightsteelblue', zorder=0)
ax.annotate(id_sta, (lon_sta-0.1, lat_sta+0.8), transform = ccrs.PlateCarree(), family = 'sans-serif')
ax.plot(lon_sta, lat_sta, transform = ccrs.PlateCarree(), color='k', alpha=0.5, linewidth=3)
plt.show()

## Dates 

In [ ]:
## Dates
start = datetime(2015, 1, 3, 0, 0, 0)
end = datetime(2015, 1, 7, 23, 59, 59)

dates = pd.date_range(start, end, freq='3h')

dates_vector = create_date_vect(dates)

## Open WAVEWATCHIII Equivalent Force Amplitude

In [ ]:
## Open Model
path_model = './F/'

## Open Synthetics Archive 

In [ ]:
## Open GF archive info
### Radial Component
fe , freq, distance, spectrum_synth_R = open_archive('../../data/spectrum_archive_tapered_R.h5')
N_time = len(time)

### Vertical Component
fe , freq, distance, spectrum_synth_Z = open_archive('../../data/spectrum_archive_tapered_Z.h5')


## Synthetic Cross-correlation Computation

In [ ]:
## Compute CCF and save it
coords_sta = [lon_sta, lat_sta]
#### Keep Correlations
corr_3h = np.zeros((len(dates_vector), N_time))
comp = 'ZN'
### Date Info
for i, date_vect in enumerate(dates_vector):
    ### CCF
    filename_ccf = './CCF/ccf_%s_%d%02d%02d%02d.nc'%(id,  date_vect[0],date_vect[1],date_vect[2],date_vect[3]) # File Name Understandable

    if os.path.exists(filename_ccf):  # Check if file already exists
        print('%s already computed'%filename_ccf)
        corr = xr.open_dataarray(filename_ccf)
    else:  # if not, compute CCF
        t1 = ttime.perf_counter()
        corr, time_corr = ccf_computation_autocorr(coords_sta, path_model, date_vect, spectrum_synth_R, spectrum_synth_Z,extent=extent, fe=fe, N=N_time, comp=comp)
        t2 = ttime.perf_counter()
        print(" Time for station pair: ", id_sta, id_sta, " elapsed: ", t2-t1)
        try:
            corr.to_netcdf('./CCF/ccf_%s_%d%02d%02d%02d.nc'%(id_sta, date_vect[0],date_vect[1],date_vect[2],date_vect[3]))
        except:
            print("make directory")
            os.mkdir('./CCF')
            corr.to_netcdf('./CCF/ccf_%s_%d%02d%02d%02d.nc'%(id_sta, date_vect[0],date_vect[1],date_vect[2],date_vect[3]))
    corr_3h[i, :] = corr.values
    # TAKES 25 MIN #

## Plot

In [ ]:
## Plot stack 

plt.figure()
plt.plot(corr.time, np.mean(corr_3h, axis=0), 'k')
plt.xlim(-200, 200)
plt.xlabel('Lag Time (s)')
plt.title('%s - %s stack'%(id_sta, id_sta))
plt.show()

## Plot CCF
normalize_corr = np.nanmax(abs(corr_3h))
dates_corr = dates

fig, ax = plt.subplots(figsize=(10,8))
im = plt.pcolormesh(corr.time, np.arange(len(dates_vector)), corr_3h/normalize_corr, cmap='bone', vmin= -1, vmax = 1)
#ax.plot(corr.time, corr_3h.T/normalize_corr + np.arange(len(dates_vector)), 'k') 
plt.colorbar(im, ax=ax)
ax.set_xlabel('Lag Time (s)')
ax.set_ylabel('Date')
ax.set_title('%s-%s CCF'%(id_sta, id_sta))
ax.set_xlim(-200, 200)
ax.set_ylim(0, len(dates_vector))
ax.set_yticks(np.arange(len(dates_vector))[::8])
ax.set_yticklabels(dates[::8].strftime('%Y-%m-%dT%H'))

plt.savefig('ccf_%s_%s_%s_synth.png'%(id_sta, id_sta, comp), dpi=300, bbox_inches='tight')